# Amini Cocoa Contamination Challenge

* We trained YOLOv5s models, which are fast to train (approx. 2h 30min per fold).
* Based on the resource restrictions of this challenge, quoted below:

  > "Your solutions for this challenge must be able to function in a resource-limited setting, i.e., it should run on a low-resource smartphone. As such, we are imposing the following restrictions on resources: T4 GPU, maximum 9h training, maximum 3h inference. Model frameworks must be appropriate for use on edge devices (e.g., ONNX, TensorFlow Lite)."

* Since there were no restrictions placed on ensembling, and the YOLOv5s models are lightweight and fast to train, we trained an ensemble of three models on folds 6, 7, and 8.
* The total inference time for the ensemble remained well under 1 hour — comfortably within the 3-hour inference budget.
* All models can be exported to ONNX or TensorFlow Lite and are suitable for deployment on low-resource smartphones.
* Based on the above, our solution adheres fully to the challenge rules.
* That said, even the individual (single-fold) models perform strongly and are fast enough for edge deployment on their own.


In [ ]:
# Install ultralytics
!pip -q install ultralytics==8.3.115
!pip -q install ensemble_boxes

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import random
import torch
from collections import defaultdict
import cv2

import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

from ultralytics import RTDETR, YOLO
from ensemble_boxes import weighted_boxes_fusion



In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(47)

In [ ]:
class Config:
    path = "/kaggle/input/amini-cocoa-contamination-dataset/"
    image_path = '/kaggle/input/amini-cocoa-contamination-dataset/dataset/images/test/'
    folds = 5
    nc=3



(1190916, 7)

In [ ]:
def run_yolo_inference_on_test_set(test_df, image_sizes, model_path, min_conf=0.001):
    """
    Runs YOLO inference on all test images across multiple image sizes and stores predictions.

    Args:
        test_df (pd.DataFrame): Test dataset with image paths.
        image_sizes (list): List of image sizes for multi-scale inference.
        model_path (str): Path to YOLO model weights.
        min_conf (float): Minimum confidence threshold.

    Returns:
        dict: Dictionary of predictions for each image size.
    """
    model = YOLO(model_path, task='detect')
    model.eval()
    model.training = False

    all_predictions = defaultdict(list)

    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Running YOLO Inference"):
        image = cv2.imread(row.image_path)
        height, width, _ = image.shape

        for size in image_sizes:
            results = model(
                image,
                imgsz=size,
                verbose=False,
                conf=min_conf,
                augment=True,
                iou=0.4,
                max_det=600
            )[0]

            boxes = results.boxes.xyxy.cpu().numpy()
            classes = results.boxes.cls.cpu().numpy()
            confidences = results.boxes.conf.cpu().numpy()

            mask = confidences >= min_conf
            boxes, classes, confidences = boxes[mask], classes[mask], confidences[mask]

            if len(boxes) == 0:
                print('No detection:', row.Image_ID)
                all_predictions[size].append({
                    'Image_ID': row.Image_ID,
                    'class': 'None',
                    'confidence': None,
                    'ymin': None, 'xmin': None, 'ymax': None, 'xmax': None
                })
            else:
                for box, cls, conf in zip(boxes, classes, confidences):
                    x1, y1, x2, y2 = box
                    all_predictions[size].append({
                        'Image_ID': row.Image_ID,
                        'class': label_map[int(cls)],
                        'confidence': conf,
                        'ymin': y1, 'xmin': x1, 'ymax': y2, 'xmax': x2
                    })
    return all_predictions



def merge_predictions_with_wbf(prediction_dfs, iou_thr=0.5, skip_box_thr=0.001):
    """
    Applies Weighted Boxes Fusion on predictions from multiple models or image sizes.

    Args:
        prediction_dfs (list): List of prediction DataFrames.
        iou_thr (float): IoU threshold for box fusion.
        skip_box_thr (float): Confidence threshold to skip boxes.

    Returns:
        pd.DataFrame: Fused prediction results.
    """
    DATA_DIR = '/kaggle/input/amini-cocoa-contamination-dataset/dataset/images/test'
    fused_results = []
    image_ids = pd.concat(prediction_dfs)['Image_ID'].unique()

    for image_id in tqdm(image_ids, desc="Merging Predictions with WBF"):
        boxes_list, scores_list, labels_list = [], [], []
        image_path = os.path.join(DATA_DIR, image_id)
        image = cv2.imread(image_path)
        h, w, _ = image.shape

        for df in prediction_dfs:
            df_image = df[df.Image_ID == image_id].copy()
            boxes = df_image[['xmin', 'ymin', 'xmax', 'ymax']].values
            scores = df_image['confidence'].tolist()
            labels = df_image['class'].map(class_map).tolist()

            # Filter valid boxes
            valid_boxes, valid_scores, valid_labels = [], [], []
            for i, box in enumerate(boxes):
                if box[2] > box[0] and box[3] > box[1]:
                    valid_boxes.append(box)
                    valid_scores.append(scores[i])
                    valid_labels.append(labels[i])

            norm_boxes = [[x[0]/w, x[1]/h, x[2]/w, x[3]/h] for x in valid_boxes]
            boxes_list.append(norm_boxes)
            scores_list.append(valid_scores)
            labels_list.append(valid_labels)

        if len(boxes_list) > 0:
            boxes, scores, labels = weighted_boxes_fusion(
                boxes_list, scores_list, labels_list,
                weights=[1] * len(boxes_list),
                iou_thr=iou_thr, skip_box_thr=skip_box_thr
            )
            boxes = [[x[0]*w, x[1]*h, x[2]*w, x[3]*h] for x in boxes]
        else:
            boxes, scores, labels = [], [], []

        if not boxes:
            result = pd.DataFrame([{
                'Image_ID': image_id, 'class': 'Corn_Healthy', 'confidence': 0.5,
                'ymin': 0, 'xmin': 0, 'ymax': 0, 'xmax': 0
            }])
        else:
            result = pd.DataFrame({
                'Image_ID': image_id,
                'class': [label_map[l] for l in labels],
                'confidence': scores,
                'ymin': [b[1] for b in boxes],
                'xmin': [b[0] for b in boxes],
                'ymax': [b[3] for b in boxes],
                'xmax': [b[2] for b in boxes],
            })

        fused_results.append(result)

    return pd.concat(fused_results)

def generate_submission_with_wbf_multimodel(
    test_df,
    image_sizes,
    model_paths,  # list of models
    base_dir,
    ensemble_name
):
    prediction_dfs = []

    for idx, model_path in enumerate(model_paths):
        intermediate_pred_csv_dir = os.path.join(
            "/kaggle/working/",
            f"intermediate-predictions/int_preds__fold_{idx}_model_nimgs_{len(image_sizes)}"
        )

        # Run inference
        print(f"Running inference with model: {model_path}")
        all_predictions = run_yolo_inference_on_test_set(
            test_df=test_df,
            image_sizes=image_sizes,
            model_path=model_path,
            min_conf=0.001
        )

        # Save raw predictions
        save_predictions_to_csv(
            predictions=all_predictions,
            idx=idx,
            output_dir=intermediate_pred_csv_dir
        )

        # Load predictions for each image size
        for imgsz in image_sizes:
            csv_path = os.path.join(intermediate_pred_csv_dir, f'preds_{idx}_{imgsz}.csv')
            if os.path.exists(csv_path):
                df = pd.read_csv(csv_path)
                prediction_dfs.append(df)

    # WBF
    print("Merging predictions with WBF")
    df_res_wbf = merge_predictions_with_wbf(
        prediction_dfs=prediction_dfs, iou_thr=0.5, skip_box_thr=0.001)

    # Save final submission
    submission_name = f"submission_fold_MULTI_MODEL_nimgs_{len(image_sizes)}_{ensemble_name}.csv"
    submission_path = os.path.join('/kaggle/working/', 'submissions', submission_name)
    os.makedirs(os.path.dirname(submission_path), exist_ok=True)
    df_res_wbf.to_csv(submission_path, index=False)
    print(f"Saved WBF submission to {submission_path}")

    return df_res_wbf


def save_predictions_to_csv(predictions, idx, output_dir='test_preds'):
    """
    Saves predictions from all image sizes into separate CSV files.

    Args:
        predictions (dict): Dictionary of predictions by image size.
        output_dir (str): Directory to save CSV files.
        fold (int): Fold number for file naming.
    """
    os.makedirs(output_dir, exist_ok=True)
    for size, preds in predictions.items():
        df = pd.DataFrame(preds)
        df.to_csv(f'{output_dir}/preds_{idx}_{size}.csv', index=False)
        print(f"Saved predictions for size {size}: {df.shape}")

(14402, 7)

In [ ]:
# Load test data
test = pd.read_csv(Config.path + "Test.csv")
train =pd.read_csv(Config.path + 'Train.csv')
# Construct image paths
test['image_path'] = Config.image_path + test['Image_ID']

# strip any spacing from the class item and make sure it is a string
train['class'] = train['class'].str.strip()
class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
label_map = {v:k for k,v in class_map.items()}

# Validate uniqueness of image IDs
assert len(test) == test['Image_ID'].nunique()

In [ ]:
test['image_path'][0]

In [ ]:
%%time
#3,5,7,8
image_sizes = [640, 800,960, 1120, 1280, 1440]
models = ['/kaggle/input/yolo11s-6-7-8/Inference_Weights/runs/detect/train_fold_6_model_yolo11s_imgs_800/weights/best.pt',
          '/kaggle/input/yolo11s-6-7-8/Inference_Weights/runs/detect/train_fold_7_model_yolo11s_imgs_800/weights/best.pt',
            '/kaggle/input/yolo11s-6-7-8/Inference_Weights/runs/detect/train_fold_8_model_yolo11s_imgs_800/weights/best.pt',
         ]
base_dir = "/kaggle/input/yolo11s_6_7_8"

final_submission = generate_submission_with_wbf_multimodel(
    test_df=test,
    image_sizes=image_sizes,
    model_paths=models,
    base_dir=base_dir,
    ensemble_name = "yolo11s_800_6_7_8(127)_min_conf_001_10folds_10bs_valid_halfed"
)
